import pandas as pd
import numpy as np 
import seaborn as sns

In [2]:
air = pd.read_csv("CHERNAIR.csv")

In [3]:
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PAYS             2051 non-null   object 
 1   Code             2051 non-null   int64  
 2   Ville            2051 non-null   object 
 3   X                2051 non-null   float64
 4   Y                2051 non-null   float64
 5   Date             2051 non-null   object 
 6   End of sampling  2051 non-null   object 
 7   Duration(h.min)  2051 non-null   float64
 8   I 131 (Bq/m3)    2031 non-null   object 
 9   Cs 134 (Bq/m3)   1897 non-null   object 
 10  Cs 137 (Bq/m3)   1571 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 176.4+ KB


What does Bq/m3 measure?
It measures the amount a nucleus is decaying. It measures the radioactivity of a substance. However, it needs to be combined with exposure to give a useful measurement. Also, check out what concentration in the air is bad.  

Also, X is lattitude and Y is longitude. 

In [4]:
air.describe()

,Code,X,Y,Duration(h.min)
count,2051.000000,2051.000000,2051.000000,2051.000000
mean,10.339834,11.960522,47.544885,20.805685
std,7.497841,11.298828,11.398712,41.351518
min,1.000000,-6.280000,0.000000,0.200000
25%,3.000000,5.180000,45.800000,6.000000
50%,8.000000,9.800000,48.460000,24.000000
75%,14.000000,14.500000,52.760000,24.000000
max,24.000000,50.680000,63.830000,700.000000


Turns out I-131 is an isotope of Iodine that can cause radioactive damage. However, high doses can be used medically. 
If we cut the knot, we can try finding out at what point these concentrations become dangerous. Also, how comparatively dangerous these are. 

In [7]:
air.Date

0       86/04/27
1       86/04/28
2       86/04/29
3       86/04/29
4       86/04/30
          ...   
2046    86/05/07
2047    86/05/08
2048    86/05/09
2049    86/05/10
2050    86/05/11
Name: Date, Length: 2051, dtype: object

Measurements are in the first few months. 

In [8]:
air.columns

Index(['PAYS', 'Code', 'Ville', 'X', 'Y', 'Date', 'End of sampling',
       'Duration(h.min)', 'I 131 (Bq/m3)', 'Cs 134 (Bq/m3)', 'Cs 137 (Bq/m3)'],
      dtype='object')

In [10]:
air.Ville.value_counts()

ISPRA                 101
KONALA(Helsinki)NW     79
CAPANNA                59
STOCKHOLM              58
VIENNA.                56
                     ... 
MARCOULE                6
WALDHOF                 6
BANSKA                  6
USTI                    5
GRAZ                    4
Name: Ville, Length: 95, dtype: int64

https://healfukushima.org/2016/02/20/iodine-129-and-131-what-are-the-hazards/
For Iodine. 

Conversion table
https://people.uwec.edu/jolhm/EH/Rosenhoeft/index5.htm

In [14]:
'KIEV' in air.Ville.values

False

In [17]:
"CHERNOBYL" in air.Ville

False

In [18]:
air.Ville.value_counts()

ISPRA                 101
KONALA(Helsinki)NW     79
CAPANNA                59
STOCKHOLM              58
VIENNA.                56
                     ... 
MARCOULE                6
WALDHOF                 6
BANSKA                  6
USTI                    5
GRAZ                    4
Name: Ville, Length: 95, dtype: int64

In [22]:
# Start with highest concentration...just want to check how bad it is compared to california infants. 
air.iloc[air['I 131 (Bq/m3)']]

ValueError: cannot mask with array containing NA / NaN values

In [20]:
air.columns

Index(['PAYS', 'Code', 'Ville', 'X', 'Y', 'Date', 'End of sampling',
       'Duration(h.min)', 'I 131 (Bq/m3)', 'Cs 134 (Bq/m3)', 'Cs 137 (Bq/m3)'],
      dtype='object')

Will want to change the isotope names to make searching easier. 

In [26]:
air['I 131 (Bq/m3)']

0             1
1        0.0046
2        0.0147
3       0.00061
4       0.00075
         ...   
2046        3.7
2047        2.2
2048       0.75
2049      0.054
2050      0.051
Name: I 131 (Bq/m3), Length: 2051, dtype: object

And, we will want to change the measurements. 

In [27]:
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PAYS             2051 non-null   object 
 1   Code             2051 non-null   int64  
 2   Ville            2051 non-null   object 
 3   X                2051 non-null   float64
 4   Y                2051 non-null   float64
 5   Date             2051 non-null   object 
 6   End of sampling  2051 non-null   object 
 7   Duration(h.min)  2051 non-null   float64
 8   I 131 (Bq/m3)    2031 non-null   object 
 9   Cs 134 (Bq/m3)   1897 non-null   object 
 10  Cs 137 (Bq/m3)   1571 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 176.4+ KB


In [28]:
air['End of sampling']

0       24:00:00
1       24:00:00
2          12:00
3       24:00:00
4       24:00:00
          ...   
2046        8:00
2047        8:00
2048        8:00
2049        8:00
2050        8:00
Name: End of sampling, Length: 2051, dtype: object

In [29]:
air['Duration(h.min)']

0       24.0
1       24.0
2       12.0
3       12.0
4       24.0
        ... 
2046    24.0
2047    24.0
2048    24.0
2049    24.0
2050    24.0
Name: Duration(h.min), Length: 2051, dtype: float64

In [30]:
air.Code

0        1
1        1
2        1
3        1
4        1
        ..
2046    24
2047    24
2048    24
2049    24
2050    24
Name: Code, Length: 2051, dtype: int64